# Energy Prediction: Data Cleaning


In [24]:
# Imports
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
data_path

'f:\\Work\\Energy Prediction\\data'

## Data reading and cleaning


In [4]:
# read csv data
df = pd.read_csv(data_path + '/raw_data/raw_energy_data.csv')
df.sample(10)


,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
151622,Mexico,Coal Tar - Non-energy uses,2012,"Metric tons, thousand",143.00,NaN,coal_tar
849247,Brazil,Natural gas liquids - Production,2008,"Metric tons, thousand",3334.00,NaN,natural_gas_liquids
221058,Colombia,Electricity - net installed capacity of electr...,2011,"Kilowatts, thousand",522.00,1.0,electricity_net_installed_capacity_of_electric...
828917,Congo,Natural gas (including LNG) - production,2012,Terajoules,8293.00,NaN,natural_gas_including_lng
107014,Germany,Brown coal - Consumption by non-metallic miner...,2013,"Metric tons, thousand",0.00,NaN,brown_coal
417238,Italy,Gas Oil/ Diesel Oil - Consumption in domestic ...,2014,"Metric tons, thousand",618.00,NaN,gas_oil_diesel_oil
18473,Niue,Aviation gasoline - Final energy consumption,2007,"Metric tons, thousand",0.02,1.0,aviation_gasoline
535713,United Kingdom,From combustible fuels – Autoproducer – Heat p...,1999,Terajoules,101436.00,NaN,heat
596670,Syrian Arab Republic,Kerosene-type Jet Fuel - International aviatio...,2010,"Metric tons, thousand",30.00,NaN,kerosene_type_jet_fuel
832181,Romania,Natural gas (including LNG) - stock changes,2001,Terajoules,-4242.00,NaN,natural_gas_including_lng


In [5]:
df.columns

Index(['country_or_area', 'commodity_transaction', 'year', 'unit', 'quantity',
       'quantity_footnotes', 'category'],
      dtype='object')

### Dropping Unnecessary Columns


In [6]:
df.isna().sum()

country_or_area                0
commodity_transaction          0
year                           0
unit                           0
quantity                       0
quantity_footnotes       1025536
category                       0
dtype: int64

The quantity_footnotes has lot of missing values. We will drop this column.


In [7]:
df = df.drop(df[['quantity_footnotes']], axis=1)
df.columns

Index(['country_or_area', 'commodity_transaction', 'year', 'unit', 'quantity',
       'category'],
      dtype='object')

### Checking Unique Values in each column


In [8]:
columns = ['country_or_area', 'commodity_transaction', 'year', 'unit', 'quantity',
           'category']

unique_countries = df['country_or_area'].unique()
unique_commodities = df['commodity_transaction'].unique()
unique_years = df['year'].unique()
unique_units = df['unit'].unique()
unique_quantity = df['quantity'].unique()
unique_categories = df['category'].unique()

print('Total rows: ', df.shape[0])
print('Unique countries: ', len(unique_countries))
print('Unique commodities: ', len(unique_commodities))
print('Unique years: ', len(unique_years))
print('Unique units: ', len(unique_units))
print('Unique quantity: ', len(unique_quantity))
print('Unique categories: ', len(unique_categories))


Total rows:  1189482
Unique countries:  243
Unique commodities:  2452
Unique years:  25
Unique units:  6
Unique quantity:  137659
Unique categories:  71


Here we will check the unique values in each column. This will help us in identifying the categorical columns and numerical columns.

`Countries: ` There are 227 countries in the dataset. We will check the countries which have the highest and lowest energy consumption.

`Commodity Transactions: ` There are 2452 unique commodity transactions. It means this column has lot more information than just the commodity name. We will need to further explore this column.

`Year: ` The data is available from 1960 to 2014. We will check the year with the highest and lowest energy consumption.

`Unit: ` There are 6 unique units. We will need to convert the units to a common unit. most prefarably to frequent unit.

`Quantity: ` The quantity column is fine as it is. We will need to convert the units to a common unit. and only need Quantity column for our analysis.

`Category: ` There are 71 unique categories. that sounds good.


### Fix Commodity Transactions


In [9]:
# get random sample of 10 rows from commodity_transaction
df['commodity_transaction'].sample(10)

211922    Electricity - net installed capacity of electr...
845519    Natural Gas (including LNG) - transformation i...
275198    Fuel oil - Consumption by other manuf., const....
118583    Brown coal - Transformation in autoproducer CH...
7262                         Anthracite - Final consumption
160289                           Coke Oven Gas - Production
827805    Natural gas (including LNG) - own use by oil r...
739425                             Motor Gasoline - Imports
665934    Liquefied petroleum gas (LPG) - Total energy s...
457187     Gas Oil/ Diesel Oil - production from refineries
Name: commodity_transaction, dtype: object

here each commodity transaction has 3 parts.

1. Commodity Name
2. Commodity Transaction
3. Additional Information

For example, the first row has the following information.

1. Commodity Name: Electricity - total
2. Commodity Transaction: Production
3. Additional Information: Including from combustible fuels

We will split the commodity transaction column into 3 columns.


In [10]:
# select commodity_transaction rows on the basis of ' commodity_name - commodity_transaction - additional_information '
df[['commodity_name', 'commodity_transaction', 'additional_information']
   ] = df['commodity_transaction'].str.split(' - ', expand=True)
df.sample(5)


,country_or_area,commodity_transaction,year,unit,quantity,category,commodity_name,additional_information
778533,Brunei Darussalam,Total energy supply,2002,"Metric tons, thousand",119.000000,naphtha,Naphtha,None
756808,Paraguay,Total energy supply,2014,"Metric tons, thousand",554.610000,motor_gasoline,Motor Gasoline,None
1069881,Paraguay,"Consumption by other manuf., const. and non-fu...",2013,"Kilowatt-hours, million",3566.950000,total_electricity,Electricity,None
1158054,Republic of Moldova,None,2011,"Kilowatt-hours, million",906.000000,total_electricity,From combustible fuels – Main activity – Elect...,None
584795,Madagascar,Final consumption,2001,"Metric tons, thousand",40.089069,kerosene_type_jet_fuel,Kerosene-type Jet Fuel,None


In [11]:
# Check unique values in new columns
unique_commodities_name = df['commodity_name'].unique()
unique_commodities_transaction = df['commodity_transaction'].unique()
unique_additional_information = df['additional_information'].unique()

print('Unique commodities name: ', len(unique_commodities_name))
print('Unique commodities transaction: ', len(unique_commodities_transaction))
print('Unique additional information: ', len(unique_additional_information))

Unique commodities name:  143
Unique commodities transaction:  320
Unique additional information:  6


These unique numbers are better than the commodity transaction column. We will use these columns for our analysis.


### Fixing Units


In [12]:
# Find most common Unit
df['unit'].value_counts()

Metric tons,  thousand     759859
Terajoules                 178937
Kilowatt-hours, million    147741
Cubic metres, thousand      52032
Kilowatts,  thousand        50229
Metric Tons                   684
Name: unit, dtype: int64

`Metric tons, thousand` is the most frequent unit. We will convert all the units to this unit.


In [13]:
quantity_mapping = {
    'Terajoules': 0.000000001,  # Convert to metric tons
    'Kilowatt-hours, million': 0.000000278,  # Convert to metric tons
    'Cubic metres, thousand': 0.000852113,  # Convert to metric tons
    'Kilowatts, thousand': 0.000000086,  # Convert to kilowatts
    'Metric Tons': 1,  # No conversion needed for the same unit
}

df['quantity'] = df['quantity'] * df['unit'].map(quantity_mapping).fillna(1)
df = df.drop(df[['unit']], axis=1)


### Save Interim Data


In [61]:
df.to_csv(data_path + '/interim_data/energy.csv', index=False)

# Global Population Cleanup


In [46]:
# import population data '/raw_data/Country-Population.xls'
columns = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963',
           '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971',
           '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
           '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
           '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
           '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
           '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
           '2012', '2013']

population = pd.read_excel(
    data_path + '/raw_data/Country-Population.xls', sheet_name='Data', usecols=columns)

population.columns


Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013'],
      dtype='object')

In [47]:
# remove 'Country Code', 'Indicator Name', 'Indicator Code',
# '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
# '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
# '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
#  '1987', '1988', '1989',

# drop columns
population = population.drop(columns=['Indicator Name', 'Indicator Code',
                                      '1960', '1961', '1962', '1963', '1964', '1965',
                                      '1966', '1967', '1968', '1969', '1970', '1971',
                                      '1972', '1973', '1974', '1975', '1976', '1977',
                                      '1978', '1979', '1980', '1981', '1982', '1983',
                                      '1984', '1985', '1986', '1987', '1988', '1989'])

# rename 'Country Name' to 'country'
population = population.rename(columns={'Country Name': 'country'})

population.columns


Index(['country', 'Country Code', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013'],
      dtype='object')

In [48]:
# save population data to csv file
population.to_csv(data_path + '/interim_data/population.csv', index=False)

# GDP Dataset


In [58]:
# import dataset
gdp = pd.read_csv(data_path + '/raw_data/GDP.csv')
gdp.columns

Index(['Country ', 'Country Code', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [59]:
# drop columns '2014', '2015', '2016', '2017', '2018', '2019'
gdp = gdp.drop(['2014', '2015', '2016', '2017', '2018', '2019'], axis=1)

gdp.columns

Index(['Country ', 'Country Code', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013'],
      dtype='object')

In [60]:
# save data to csv file
gdp.to_csv(data_path + '/interim_data/gdp.csv', index=False)

# Global Inflation DataSet


In [75]:
# import dataset
gInf = pd.read_csv(
    data_path + '/raw_data/Global_Dataset_of_Inflation.csv', encoding='latin-1')
gInf.columns

Index(['Country Code', 'IMF Country Code', 'Country', 'Indicator Type',
       'Series Name', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', 'Note', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61',
       'Unnamed: 62', 'Unnamed: 63'],
      dtype='object')

In [76]:
# only keep with Series Name: 'Energy Consumer Price Inflation'

gInf = gInf[gInf['Series Name'] == 'Energy Consumer Price Inflation']

gInf = gInf.drop(['Country Code', 'IMF Country Code', 'Series Name', 'Indicator Type', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
                  '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
                  '1986', '1987', '1988', '1989', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
                  '2022', 'Note', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61',
                  'Unnamed: 62', 'Unnamed: 63'], axis=1)

gInf


,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
203,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.08,6.69,7.43,5.51,13.79,-8.20,9.43,7.16,0.810,-7.460
204,Afghanistan,66.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.46,3.31,8.41,2.48,8.65,12.36,11.810,8.800
205,Albania,NaN,NaN,NaN,NaN,NaN,24.5,10.8,25.1,27.8,...,27.20,25.00,5.70,7.10,3.87,1.75,3.62,2.76,0.900,0.240
206,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.17,0.45,-0.30,-2.42,-2.580,0.250
207,Argentina,2861.9,178.0,24.4,18.9,-4.1,6.8,3.6,1.0,-0.1,...,0.80,4.20,-2.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,South Africa,15.4,12.3,12.6,15.7,9.6,7.9,7.1,7.8,8.2,...,-10.64,0.60,4.40,11.15,12.18,7.91,6.69,6.63,6.190,5.600
371,Zambia,92.3,212.0,121.8,270.5,56.0,41.9,41.3,35.8,29.7,...,NaN,NaN,NaN,NaN,NaN,NaN,18.24,12.45,5.180,8.440
372,Zimbabwe,9.5,41.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.42,7.99,4.46,11.800,4.470
373,British Virgin Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.100


In [77]:
# save data to csv file
gInf.to_csv(data_path + '/interim_data/globalInflation.csv', index=False)